In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=1)

import missingno as msno    # 누락값 표시
import warnings
warnings.filterwarnings('ignore')  # dnjsld antl

%matplotlib inline    
# 새로운 창 안 뜨고 노트북에 바로 뜸

In [6]:
data = pd.read_csv('./Titanic_data/train.csv')
df_test = pd.read_csv('./Titanic_data/test.csv')
submission = pd.read_csv('./Titanic_data/gender_submission.csv')

# Continuous into Categorical: Binning or Nomalisation

In [9]:
# Age_band

data['Age_band']=0
data.loc[data['Age']<=16, 'Age_band']=0
data.loc[(16<data['Age']) & (data['Age']<=32), 'Age_band']=1
data.loc[(32<data['Age']) & (data['Age']<=48), 'Age_band']=2
data.loc[(48<data['Age']) & (data['Age']<=64), 'Age_band']=3
data.loc[(64<data['Age']) & (data['Age']<=80), 'Age_band']=4

# sns.factorplot('Age_band', 'Survived', data=data, col='Pclass')
data.groupby('Pclass').count()
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_band
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,2


In [10]:
df_test['Age_band']=0
df_test.loc[df_test['Age']<=16, 'Age_band']=0
df_test.loc[(16<df_test['Age']) & (df_test['Age']<=32), 'Age_band']=1
df_test.loc[(32<df_test['Age']) & (df_test['Age']<=48), 'Age_band']=2
df_test.loc[(48<df_test['Age']) & (df_test['Age']<=64), 'Age_band']=3
df_test.loc[(64<df_test['Age']) & (df_test['Age']<=80), 'Age_band']=4

In [12]:
# Family_size and Alone

data['Family_size']=0
data['Family_size']= data['Parch']+data['SibSp']
data['Alone']=0
data.loc[data['Family_size']==0, 'Alone']=1

sns.factorplot('Family_size', 'Survived',data=data)
sns.factorplot('Alone', 'Survived',data=data)

AttributeError: module 'seaborn' has no attribute 'factorplot'

In [ ]:
df_test['Family_size']=0
df_test['Family_size']= df_test['Parch']+df_test['SibSp']
df_test['Alone']=0
df_test.loc[df_test['Family_size']==0, 'Alone']=1

In [ ]:
# Fare_range
data['Fare_range']=pd.qcut(data.Fare, 4)
data.groupby('Fare_range').Survived.mean().to_frame()


data.Fare_cat=0
data.loc[ data['Fare']<=7.91,'Fare_cat']=0
data.loc[(data['Fare']>7.91) & (data['Fare']<=14.454), 'Fare_cat']=1
data.loc[(data['Fare']>14.454) & (data['Fare']<=31.0), 'Fare_cat']=2
data.loc[(data['Fare']>31.0) & (data['Fare']<=512.329), 'Fare_cat']=3

sns.factorplot('Fare_cat', 'Survived', data=data, hue='Sex')

In [ ]:
df_test['Fare_range']=pd.qcut(df_test.Fare, 4)

df_test.Fare_cat=0
df_test.loc[ df_test['Fare']<=7.91,'Fare_cat']=0
df_test.loc[(df_test['Fare']>7.91) & (df_test['Fare']<=14.454), 'Fare_cat']=1
df_test.loc[(df_test['Fare']>14.454) & (df_test['Fare']<=31.0), 'Fare_cat']=2
df_test.loc[(df_test['Fare']>31.0) & (df_test['Fare']<=512.329), 'Fare_cat']=3

# Converting String Values into Numeric



In [ ]:
data.Sex.replace(['male', 'female'],[0,1], inplace=True)
data.Embarked.replace(['S','C','Q'], [0,1,2], inplace=True)

In [ ]:
df_test.Sex.replace(['male', 'female'],[0,1], inplace=True)
df_test.Embarked.replace(['S','C','Q'], [0,1,2], inplace=True)

In [ ]:
data.drop(['Name', 'Cabin', 'Ticket', 'Fare', 'Fare_range', 'Age', 'PassengerId'], axis=1, inplace=True)

In [ ]:
df_test.drop(['Name', 'Cabin', 'Ticket', 'Fare', 'Fare_range', 'Age', 'PassengerId'], axis=1, inplace=True)

In [ ]:
sns.heatmap(data.corr(), annot=True, cmap='RdYlGn', linewidths=0.2, annot_kws={'size':20})
fig=plt.gcf()
fig.set_size_inches(20,15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

In [ ]:
sns.heatmap(df_test.corr(), annot=True, cmap='RdYlGn', linewidths=0.2, annot_kws={'size':20})
fig=plt.gcf()
fig.set_size_inches(20,15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Part 3: Predictive Modeling

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [ ]:
train,test= train_test_split(data, test_size=0.3, random_state=0, stratify=data['Survived'])
train_X=train[train.columns[1:]]  # 1
train_Y=train[train.columns[:1]]  # 0
test_X=test[test.columns[1:]]     # 1
test_Y=test[test.columns[:1]]     # 0
X=data[data.columns[1:]]
Y=data['Survived']

data.loc[data['Fare_cat'].isnull(),'Fare_cat']=0
data.loc[data['Embarked'].isnull(),'Embarked']=0
data.isnull().any()

In [ ]:
data

In [ ]:
# Radial Support Vector Machines(rbf-SVM)
model=svm.SVC(kernel='rbf', C=1, gamma=0.1)
model.fit(train_X, train_Y)
prediction1=model.predict(test_X)
print('Accuracy for rbf SVM is', metrics.accuracy_score(prediction1, test_Y))

# Linear Support Vector Machine(linear-SVM)
model=svm.SVC(kernel='linear', C=0.1, gamma=0.1)
model.fit(train_X, train_Y)
prediction2= model.predict(test_X)
print('Accuracy for linear SVM is', metrics.accuracy_score(prediction2, test_Y))

# Logistic Regression
model= LogisticRegression()
model.fit(train_X, train_Y)
prediction3= model.predict(test_X)
print('The accuracy of the Logistic Regression is', metrics.accuracy_score(prediction3, test_Y))

# Decision Tree
model= DecisionTreeClassifier()
model.fit(train_X, train_Y)
prediction4= model.predict(test_X)
print('The accuracy of the Decision Tree is', metrics.accuracy_score(prediction4, test_Y))

# K-Nearest Neighbors(KNN)
model= KNeighborsClassifier()
model.fit(train_X, train_Y)
prediction5= model.predict(test_X)
print('The accuracy of the KNN is', metrics.accuracy_score(prediction5, test_Y))  # n_neighbors= 5일 때 점수
'''
a=pd.Series()
a_index= list(range(1,11))
for i in list(range(1,11)):
  model= KNeighborsClassifier(n_neighbors=i)
  model.fit(train_X, train_Y)
  prediction= model.predict(test_X)
  a= a.append(pd.Series(metrics.accuracy_score(prediction, test_Y)))
plt.plot(a_index,a)
plt.xticks(list(range(0,11)))
fig=plt.gcf()
fig.set_size_inches(12,6)
plt.show()
'''
# Gaussian Naive Bayes
model= GaussianNB()
model.fit(train_X, train_Y)
prediction6= model.predict(test_X)
print('The accuracy of the NaiveBayes is', metrics.accuracy_score(prediction6, test_Y))

# Random Forests
model= RandomForestClassifier(n_estimators=100)
model.fit(train_X, train_Y)
prediction7= model.predict(test_X)
print('The accuracy of the Random Forests is', metrics.accuracy_score(prediction7, test_Y))

# Cross Validation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

kfold= KFold(n_splits=10, random_state=22, shuffle=True)
xyz=[]
accuracy=[]
std=[]
classifiers=['Linear Svm', 'Radial Svm', 'Logistic Regression', 'KNN', 'Decision Tree', 'Naive Bayes', 'Random Forest']
models=[svm.SVC(kernel='linear'), svm.SVC(kernel= 'rbf'), LogisticRegression(), KNeighborsClassifier(n_neighbors=9), DecisionTreeClassifier(),
        GaussianNB(), RandomForestClassifier(n_estimators=100)]
Y=data['Survived']
for i in models:
  model = i
  cv_result= cross_val_score(model,X,Y, cv= kfold, scoring= 'accuracy')
  cv_result=cv_result
  xyz.append(cv_result.mean())
  std.append(cv_result.std())
  accuracy.append(cv_result)
new_models_dataframe2= pd.DataFrame({'CV Mean': xyz, 'Std': std}, index=classifiers)
new_models_dataframe2

In [ ]:
plt.subplots(figsize=(12,6))
box=pd.DataFrame(accuracy, index=(classifiers))
plt.yticks(np.arange(0.70,0.9,0.02))
box.T.boxplot()

In [ ]:
new_models_dataframe2['CV Mean'].plot.barh(width=0.8)
fig=plt.gcf()
plt.title('Average CV Mean Accuracy')
fig.set_size_inches(8,5)
plt.show()

# Confusion Matrix

In [ ]:
f, ax= plt.subplots(3,3, figsize=(12,10))
y_pred= cross_val_predict(svm.SVC(kernel='rbf'), X,Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,0], annot=True, fmt='2.0f')
ax[0,0].set_title('Matrix for rbf-SVM')
y_pred= cross_val_predict(svm.SVC(kernel='linear'), X,Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,1], annot=True, fmt='2.0f')
ax[0,1].set_title('Matrix for linear-SVM')
y_pred= cross_val_predict(KNeighborsClassifier(n_neighbors=9), X,Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,2], annot=True, fmt='2.0f')
ax[0,2].set_title('Matrix for KNN')
y_pred= cross_val_predict(RandomForestClassifier(n_estimators=100), X,Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,0], annot=True, fmt='2.0f')
ax[1,0].set_title('Matrix for Random-Forests')
y_pred= cross_val_predict(LogisticRegression(), X,Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,1], annot=True, fmt='2.0f')
ax[1,1].set_title('Matrix for Logistic Regression')
y_pred= cross_val_predict(DecisionTreeClassifier(), X,Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,2], annot=True, fmt='2.0f')
ax[1,2].set_title('Matrix for Decision Tree')
y_pred= cross_val_predict(GaussianNB(), X,Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[2,0], annot=True, fmt='2.0f')
ax[2,0].set_title('Matrix for Naive Bayse')
plt.subplots_adjust(hspace=0.2, wspace=0.2)
plt.show()

# SVM-rbf는 사망자 정확도가, Naive Bayse는 생존자 정확도가 제일 높다.

# Hyper-Parameters Tuning

In [ ]:
# SVM

from sklearn.model_selection import GridSearchCV
C=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
gamma=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
kernel=['rbf','linear']
hyper={'kernel':kernel, 'C':C, 'gamma':gamma}
gd=GridSearchCV(estimator=svm.SVC(), param_grid=hyper, verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

In [ ]:
# Random Forest

n_estimators=range(100,1001,100)
hyper={'n_estimators':n_estimators}
gd=GridSearchCV(estimator=RandomForestClassifier(random_state=0), param_grid=hyper, verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

# Ensembling

In [ ]:
# Voting Classfier

from sklearn.ensemble import VotingClassifier

ensemble_lin_rbf=VotingClassifier(estimators=[('KNN', KNeighborsClassifier(n_neighbors=10)),
                                              ('RBF', svm.SVC(probability=True, kernel='rbf', C=0.9, gamma=0.2)),
                                              ('RFor', RandomForestClassifier(n_estimators=300, random_state=0)),
                                              ('LR', LogisticRegression(C=0.05)),
                                              ('DT', DecisionTreeClassifier(random_state=0)),
                                              ('NB', GaussianNB()),
                                              ('svm', svm.SVC(kernel='linear', probability=True))],
                                  voting='soft').fit(train_X,train_Y)
print('The accuracy for ensembled model is:', ensemble_lin_rbf.score(test_X,test_Y))
cross=cross_val_score(ensemble_lin_rbf,X,Y, cv=10, scoring='accuracy')
print('The cross validated score is', cross.mean())                               

In [ ]:
# Bagging

from sklearn.ensemble import BaggingClassifier
model= BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=3), random_state=0, n_estimators=700)
model.fit(train_X,train_Y)
prediction=model.predict(test_X)
print('The accuracy for bagged KNN is:', metrics.accuracy_score(prediction,test_Y))
result=cross_val_score(model,X,Y,cv=10, scoring='accuracy')
print('The cross validation score for bagged KNN is:', result.mean())

In [ ]:
model=BaggingClassifier(base_estimator=DecisionTreeClassifier(), random_state=0, n_estimators=100)
model.fit(train_X,train_Y)
prediction=model.predict(test_X)
print('The accuracy for bagged Decision Tree is:', metrics.accuracy_score(prediction,test_Y))
result=cross_val_score(model,X,Y,cv=10, scoring='accuracy')
print('The cross validation score for Decision Tree is:', result.mean())


In [ ]:
# Boosting

# AdaBoost(Adaptive Boosting)
from sklearn.ensemble import AdaBoostClassifier
ada=AdaBoostClassifier(n_estimators=200, random_state=0, learning_rate=0.1)
result=(ada,X,Y,cv=10, scoring='accuracy')
print('The cross validated score for AdaBoost is:', result.mean())

In [ ]:
# Stochastic Gradient Boosting

from sklearn.ensemble import GradientBoostingClassifier
grad=GradientBoostingClassifier(n_estimators=500, random_state=0, learning_rate=0.1)
result=cross_val_score(grad,X,Y,cv=10, scoring='accuracy')
print('The cross validated score for Gradient Boosting is:', result.mean())

In [ ]:
# XGBoosting

import xgboost as xg
xgboost=xg.XGBClassifier(n_estimators=900, learning_rate=0.1)
result=cross_val_score(xgboost,X,Y,cv=10, scoring='accuracy')
print('The cross validated score for XGBoost is:', result.mean())

In [ ]:
# Hyper-Parameter Tuning for XGBoosting

n_estimators=range(100,1001,100)
learning_rate=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
hyper={'n_estimators':n_estimators, 'learning_rate':learning_rate}
gd=GridSearchCV(estimator=AdaBoostClassifier(),param_grid=hyper,verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

In [ ]:
# Confusion Matrix for the Best Model

f,ax=plt.subplots(1,2,figsize=(10,5))
xgboost=xgboost=xg.XGBClassifier(n_estimators=100, learning_rate=0.4)
result=cross_val_predict(xgboost,X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,result),annot=True,fmt='2.0f',ax=ax[0])
grad=GradientBoostingClassifier(n_estimators=400, random_state=0, learning_rate=0.3)
result1=cross_val_predict(grad,X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,result1),annot=True,fmt='2.0f',ax=ax[1])
plt.show()

# Feature Importance: 어떤 Feature를 가장 중요하게 여기나

In [ ]:
f,ax=plt.subplots(2,2, figsize=(15,12))

model=RandomForestClassifier(n_estimators=900,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[0,0])
ax[0,0].set_title('Feature Importance in Random Forests')

model=AdaBoostClassifier(n_estimators=100,learning_rate=0.1, random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[0,1])
ax[0,1].set_title('Feature Importance in AdaBoost')

model=GradientBoostingClassifier(n_estimators=400,learning_rate=0.3, random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,0])
ax[1,0].set_title('Feature Importance in Gradient Boosting')

model=xg.XGBClassifier(n_estimators=100,learning_rate=0.4, random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,1])
ax[1,1].set_title('Feature Importance in XGB')

In [ ]:
submission.head()
test['Survived']=prediction
test.Survived.head()

In [ ]:
test.drop('Survived', axis=1, inplace=True)

In [ ]:
A[A['Fare_cat'].isnull()]=1
A.isnull().any()

In [ ]:
submission1.to_csv('/content/drive/MyDrive/Titanic/Titanic_data/submission1.csv', index=False)